In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
from tqdm.auto import tqdm
from joblib import delayed, Parallel
from os import remove
from qiskit.test.mock import FakeMontreal

from grouping import Grouping
from utils import tqdm_joblib, random_labels

In [3]:
n = 50  # Number of strings
N = 15  # Number of qubits
labels = random_labels(n, N)

In [4]:
def n_groups(file_name):
    groups = Grouping(load=file_name)
    
    groups._shuffle_labels()
    groups._shuffle_qubits()
    
    groups.group()
    
    return groups.n_groups, groups.grouping_time

def parallel_shuffling(labels, transpiled_order, connected_graph, name_algoritm, n_reps=100):
    file_name_temp = 'tmp_grouping'
    grouping = Grouping(labels, WC_device, print_progress=False, transpiled_order=transpiled_order,
                                connected_graph=connected_graph, tests=None, method='HEEM')
    grouping.save_data(file_name_temp)

    pbar = tqdm(range(n_reps), desc='Shuffling Pauli and qubits order ({})'.format(name_algoritm))
    with tqdm_joblib(pbar) as _:
        results = Parallel(n_jobs=-1)(delayed(n_groups)(file_name_temp) for _ in range(n_reps))
    
    remove(file_name_temp + '.pickle')
    
    n_groups_list = [result[0] for result in results]
    time = [result[1] for result in results]
    
    data = {'n_groups': n_groups_list, 'time': time}    
    return data

In [5]:
backend = FakeMontreal()
WC_device = backend.configuration().coupling_map
WC_device = [tuple(x) for x in WC_device]

In [6]:
def compare_algorithms(labels, n_reps=100):
    data = {}
    
    # Naive
    data['naive'] = parallel_shuffling(labels, transpiled_order=False, connected_graph=None, name_algoritm='naive', n_reps=n_reps)
    
    # Disconnected
    data['disconnected'] = parallel_shuffling(labels, transpiled_order=True, connected_graph=False, name_algoritm='disconnected', n_reps=n_reps)
    
    # Connected
    data['connected'] = parallel_shuffling(labels, transpiled_order=True, connected_graph=True, name_algoritm='connected', n_reps=n_reps)
    
    return data

In [8]:
data = compare_algorithms(labels, n_reps=2000)

Shuffling Pauli and qubits order (naive):   0%|          | 0/2000 [00:00<?, ?it/s]

Shuffling Pauli and qubits order (disconnected):   0%|          | 0/2000 [00:00<?, ?it/s]

Shuffling Pauli and qubits order (connected):   0%|          | 0/2000 [00:00<?, ?it/s]

In [14]:
print(np.median(data['naive']['n_groups']), np.median(data['disconnected']['n_groups']), np.median(data['connected']['n_groups']))

37.0 37.0 35.0
